Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-parameter-tuning-with-hyperdrive.png)

# Azure Machine Learning Pipeline with HyperDriveStep


This notebook is used to demonstrate the use of HyperDriveStep in AML Pipeline.

## Prerequisites and Azure Machine Learning Basics
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration Notebook](https://aka.ms/pl-config) first if you haven't. This sets you up with a working config file that has information on your workspace, subscription id, etc. 

## Azure Machine Learning and Pipeline SDK-specific imports

In [6]:
import azureml.core
from azureml.core import Workspace, Experiment
from azureml.core.datastore import Datastore
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.environment import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.exceptions import ComputeTargetException
from azureml.data.data_reference import DataReference
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.train.dnn import TensorFlow
# from azureml.train.hyperdrive import *
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

import os
import shutil
import urllib
import numpy as np
# import matplotlib.pyplot as plt

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.1.5


## Initialize workspace

Initialize a workspace object from persisted configuration. If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure the config file is present at .\config.json

In [8]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

avadevitsmlsvc
RG-ITSMLTeam-Dev
westus2
ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e


## Create an Azure ML experiment
Let's create an experiment named "tf-mnist" and a folder to hold the training scripts. 

> The best practice is to use separate folders for scripts and its dependent files for each step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step. 

> The script runs will be recorded under the experiment in Azure.

In [9]:
script_folder = './tf-mnist'
os.makedirs(script_folder, exist_ok=True)

exp = Experiment(workspace=ws, name='Hyperdrive_sample')

## Download MNIST dataset
In order to train on the MNIST dataset we will first need to download it from Yan LeCun's web site directly and save them in a `data` folder locally.

In [10]:
os.makedirs('./data/mnist', exist_ok=True)

urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', filename = './data/mnist/train-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', filename = './data/mnist/train-labels.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', filename = './data/mnist/test-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', filename = './data/mnist/test-labels.gz')

('./data/mnist/test-labels.gz', <http.client.HTTPMessage at 0x163c2714fc8>)

## Show some sample images
Let's load the downloaded compressed file into numpy arrays using some utility functions included in the `utils.py` library file from the current folder. Then we use `matplotlib` to plot 30 random images from the dataset along with their labels.

In [12]:
from utils import load_data

# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the neural network converge faster.
X_train = load_data('./data/mnist/train-images.gz', False) / 255.0
y_train = load_data('./data/mnist/train-labels.gz', True).reshape(-1)

X_test = load_data('./data/mnist/test-images.gz', False) / 255.0
y_test = load_data('./data/mnist/test-labels.gz', True).reshape(-1)

# count = 0
# sample_size = 30
# plt.figure(figsize = (16, 6))
# for i in np.random.permutation(X_train.shape[0])[:sample_size]:
#     count = count + 1
#     plt.subplot(1, sample_size, count)
#     plt.axhline('')
#     plt.axvline('')
#     plt.text(x = 10, y = -10, s = y_train[i], fontsize = 18)
#     plt.imshow(X_train[i].reshape(28, 28), cmap = plt.cm.Greys)
# plt.show()

## Upload MNIST dataset to blob datastore 
A [datastore](https://docs.microsoft.com/azure/machine-learning/service/how-to-access-data) is a place where data can be stored that is then made accessible to a Run either by means of mounting or copying the data to the compute target. In the next step, we will use Azure Blob Storage and upload the training and test set into the Azure Blob datastore, which we will then later be mount on a Batch AI cluster for training.

In [13]:
ds = ws.get_default_datastore()
ds.upload(src_dir='./data/mnist', target_path='mnist', overwrite=True, show_progress=True)

Uploading an estimated of 4 files
Uploading ./data/mnist\test-images.gz
Uploading ./data/mnist\test-labels.gz
Uploading ./data/mnist\train-images.gz
Uploading ./data/mnist\train-labels.gz
Uploaded ./data/mnist\test-labels.gz, 1 files out of an estimated total of 4
Uploaded ./data/mnist\train-labels.gz, 2 files out of an estimated total of 4
Uploaded ./data/mnist\test-images.gz, 3 files out of an estimated total of 4
Uploaded ./data/mnist\train-images.gz, 4 files out of an estimated total of 4
Uploaded 4 files


$AZUREML_DATAREFERENCE_f5c13e50314544f9a045e3d1876f2890

## Retrieve or create a Azure Machine Learning compute
Azure Machine Learning Compute is a service for provisioning and managing clusters of Azure virtual machines for running machine learning workloads. Let's create a new Azure Machine Learning Compute in the current workspace, if it doesn't already exist. We will then run the training script on this compute target.

If we could not find the compute with the given name in the previous cell, then we will create a new compute here. This process is broken down into the following steps:

1. Create the configuration
2. Create the Azure Machine Learning compute

**This process will take a few minutes and is providing only sparse output in the process. Please make sure to wait until the call returns before moving to the next cell.**


In [14]:
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target {}.'.format(cluster_name))
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_NC6",
                                                               max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)

print("Azure Machine Learning Compute attached")

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Azure Machine Learning Compute attached


## Copy the training files into the script folder
The TensorFlow training script is already created for you. You can simply copy it into the script folder, together with the utility library used to load compressed data file into numpy array.

In [15]:
# the training logic is in the tf_mnist.py file.
shutil.copy('./tf_mnist.py', script_folder)

# the utils.py just helps loading data from the downloaded MNIST dataset into numpy arrays.
shutil.copy('./utils.py', script_folder)

'./tf-mnist\\utils.py'

## Create TensorFlow estimator
Next, we construct an [TensorFlow](https://docs.microsoft.com/python/api/azureml-train-core/azureml.train.dnn.tensorflow?view=azure-ml-py) estimator object.
The TensorFlow estimator is providing a simple way of launching a TensorFlow training job on a compute target. It will automatically provide a docker image that has TensorFlow installed -- if additional pip or conda packages are required, their names can be passed in via the `pip_packages` and `conda_packages` arguments and they will be included in the resulting docker.

The TensorFlow estimator also takes a `framework_version` parameter -- if no version is provided, the estimator will default to the latest version supported by AzureML. Use `TensorFlow.get_supported_versions()` to get a list of all versions supported by your current SDK version or see the [SDK documentation](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.dnn?view=azure-ml-py) for the versions supported in the most current release.

The TensorFlow estimator also takes a `framework_version` parameter -- if no version is provided, the estimator will default to the latest version supported by AzureML. Use `TensorFlow.get_supported_versions()` to get a list of all versions supported by your current SDK version or see the [SDK documentation](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.dnn?view=azure-ml-py) for the versions supported in the most current release.

In [16]:
est = TensorFlow(source_directory=script_folder,                 
                 compute_target=compute_target,
                 entry_script='tf_mnist.py', 
                 use_gpu=True,
                 framework_version='1.13')

## Intelligent hyperparameter tuning
Now let's try hyperparameter tuning by launching multiple runs on the cluster. First let's define the parameter space using random sampling.

In this example we will use random sampling to try different configuration sets of hyperparameters to maximize our primary metric, the best validation accuracy (`validation_acc`).

In [17]:
ps = RandomParameterSampling(
    {
        '--batch-size': choice(25, 50, 100),
        '--first-layer-neurons': choice(10, 50, 200, 300, 500),
        '--second-layer-neurons': choice(10, 50, 200, 500),
        '--learning-rate': loguniform(-6, -1)
    }
)

Now we will define an early termnination policy. The `BanditPolicy` basically states to check the job every 2 iterations. If the primary metric (defined later) falls outside of the top 10% range, Azure ML terminate the job. This saves us from continuing to explore hyperparameters that don't show promise of helping reach our target metric.

Refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-tune-hyperparameters#specify-an-early-termination-policy) for more information on the BanditPolicy and other policies available.

In [18]:
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

Now we are ready to configure a run configuration object, and specify the primary metric `validation_acc` that's recorded in your training runs. If you go back to visit the training script, you will notice that this value is being logged after every epoch (a full batch set). We also want to tell the service that we are looking to maximizing this value. We also set the number of samples to 20, and maximal concurrent job to 4, which is the same as the number of nodes in our computer cluster.

In [19]:
hd_config = HyperDriveConfig(estimator=est, 
                             hyperparameter_sampling=ps,
                             policy=early_termination_policy,
                             primary_metric_name='validation_acc', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=4,
                             max_concurrent_runs=4)

## Add HyperDrive as a step of pipeline

### Setup an input for the hypderdrive step
Let's setup a data reference for inputs of hyperdrive step.

In [20]:
data_folder = DataReference(
    datastore=ds,
    data_reference_name="mnist_data")

### HyperDriveStep
HyperDriveStep can be used to run HyperDrive job as a step in pipeline.
- **name:** Name of the step
- **hyperdrive_config:** A HyperDriveConfig that defines the configuration for this HyperDrive run
- **estimator_entry_script_arguments:** List of command-line arguments for estimator entry script
- **inputs:** List of input port bindings
- **outputs:** List of output port bindings
- **metrics_output:** Optional value specifying the location to store HyperDrive run metrics as a JSON file
- **allow_reuse:** whether to allow reuse
- **version:** version


In [21]:
metrics_output_name = 'metrics_output'
metrics_data = PipelineData(name='metrics_data',
                             datastore=ds)

hd_step_name='hd_step01'
hd_step = HyperDriveStep(
    name=hd_step_name,
    hyperdrive_config=hd_config,
    estimator_entry_script_arguments=['--data-folder', data_folder],
    inputs=[data_folder],
    metrics_output=metrics_data,
    allow_reuse=True)

In [22]:
conda_run_config = RunConfiguration(framework="python")
conda_run_config.target = compute_target
cd = CondaDependencies.create(pip_packages=['pandas', 'azureml-defaults'], 
                              pin_sdk_version=True)
conda_run_config.environment.python.conda_dependencies = cd

best_run_data = PipelineData('best_run_data', is_directory=True, datastore=ds)

best_run_step = PythonScriptStep(
    name='get best run',
    script_name='metrics.py',
    compute_target=compute_target,
        arguments=['--input_file', metrics_data,
               '--output_dir', best_run_data],
    inputs=[metrics_data],
    outputs=[best_run_data],
    runconfig=conda_run_config,
    source_directory=os.path.join(os.getcwd(), 'metrics'),
    allow_reuse=True
)

### Run the pipeline

In [23]:
pipeline = Pipeline(workspace=ws, steps=[best_run_step])
pipeline_run = exp.submit(pipeline)

Created step get best run [31954077][a9842d81-7995-43c1-bb55-df20aa92e1b0], (This step will run and generate new outputs)
Created step hd_step01 [7c9d2beb][d203a8c7-27ce-4ebf-8afc-30ef240e2a0f], (This step will run and generate new outputs)
Using data reference mnist_data for StepId [f338e5e0][c34f1c52-35c4-4553-a4fd-0ffa20b750c6], (Consumers of this data are eligible to reuse prior runs.)
Submitted PipelineRun 5e453cc5-031b-45b6-8144-1042c2ce1c66
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Hyperdrive_sample/runs/5e453cc5-031b-45b6-8144-1042c2ce1c66?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc


### Monitor using widget

### Wait for the completion of this Pipeline run

In [25]:
pipeline_run.wait_for_completion()

6: Preparing
ccdb13a20bf2: Preparing
9513cdf4e497: Preparing
7f083f9454c0: Preparing
29f36b5893dc: Preparing
2af19136c6e4: Waiting
e1171d4d60ca: Waiting
6ef1a8ae63b7: Waiting
85389f9ead9e: Waiting
f2608f66a0e3: Waiting
0e259b09e5f4: Waiting
340dc32eb998: Waiting
df18b66efaa6: Waiting
ccdb13a20bf2: Waiting
9513cdf4e497: Waiting
7f083f9454c0: Waiting
29f36b5893dc: Waiting
5cc9db7391c3: Pushed
9839507b53e9: Pushed
6ada5d42555e: Pushed
2af19136c6e4: Pushed
8d30a00f5b79: Pushed
e1171d4d60ca: Pushed
6ef1a8ae63b7: Pushed

340dc32eb998: Pushed
0e259b09e5f4: Pushed
ccdb13a20bf2: Pushed
9513cdf4e497: Pushed
f2608f66a0e3: Pushed
7f083f9454c0: Pushed
85389f9ead9e: Pushed

29f36b5893dc: Pushed
df18b66efaa6: Pushed
63a5f7d0d524: Pushed
latest: digest: sha256:39923ba32e36cde922f2711479b1c0c01bbce539bd44311524f85a85d1c6e5b2 size: 3883
2020/03/20 22:30:30 Successfully pushed image: avadevitsmlsvc4326118371.azurecr.io/azureml/azureml_3f9cdf0ac8964f43b11558b4296c26fc:latest
2020/03/20 22:30:30 Step ID: a

'Finished'

Note the

`Created step hd_step01 [7c9d2beb][d203a8c7-27ce-4ebf-8afc-30ef240e2a0f], (This step will run and generate new outputs)`

In [26]:
pipeline = Pipeline(workspace=ws, steps=[best_run_step])
pipeline_run = exp.submit(pipeline)

Created step get best run [31c67e94][a9842d81-7995-43c1-bb55-df20aa92e1b0], (This step is eligible to reuse a previous run's output)
Created step hd_step01 [54aa5211][a10bf668-d90f-44a4-9524-8759d419aa95], (This step will run and generate new outputs)
Using data reference mnist_data for StepId [e4f91bb0][c34f1c52-35c4-4553-a4fd-0ffa20b750c6], (Consumers of this data are eligible to reuse prior runs.)
Submitted PipelineRun 4ab4b3b4-c6c4-424b-9cb2-4fc041ace64f
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Hyperdrive_sample/runs/4ab4b3b4-c6c4-424b-9cb2-4fc041ace64f?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc


Note its the same story again

`Created step hd_step01 [54aa5211][a10bf668-d90f-44a4-9524-8759d419aa95], (This step will run and generate new outputs)`

In [27]:
pipeline_run.wait_for_completion(show_output=True)

 Running


StepRunId: 966635a6-227c-48b4-8a79-14bfc751dddd
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Hyperdrive_sample/runs/966635a6-227c-48b4-8a79-14bfc751dddd?wsid=/subscriptions/ff2e23ae-7d7c-4cbd-99b8-116bb94dca6e/resourcegroups/RG-ITSMLTeam-Dev/workspaces/avadevitsmlsvc
StepRun( hd_step01 ) Status: NotStarted
StepRun( hd_step01 ) Status: Running

StepRun(hd_step01) Execution Summary
StepRun( hd_step01 ) Status: Finished
{'runId': '966635a6-227c-48b4-8a79-14bfc751dddd', 'status': 'Completed', 'startTimeUtc': '2020-03-20T22:39:23.978593Z', 'endTimeUtc': '2020-03-20T22:49:51.892284Z', 'properties': {'azureml.runsource': 'azureml.StepRun', 'ContentSnapshotId': 'fc1a41b8-5804-42db-9886-92784784c69e', 'StepType': 'HyperDriveStep', 'ComputeTargetType': 'HyperDrive', 'azureml.pipelinerunid': '4ab4b3b4-c6c4-424b-9cb2-4fc041ace64f'}, 'inputDatasets': [], 'logFiles': {'logs/azureml/executionlogs.txt': 'https://avadevitsmlsvc7139228118.blob.core.windows.net/azure

'Finished'